In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.sprint_lap_time`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

In [4]:
df_bq=df_bq2[(df_bq2["GP"]=='United States Grand Prix')&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_27833/1219114623.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GP["LapNumber"]=GP["LapNumber"].astype(int)


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
349,0 days 00:47:55.684000,ALB,23,NaT,1,1.0,0 days 00:46:15.024000,NaT,NaT,0 days 00:00:40.701000,0 days 00:00:34.074000,NaT,0 days 00:47:21.680000,0 days 00:47:55.898000,209.0,174.0,206.0,317.0,False,MEDIUM,1.0,True,Williams,0 days 00:46:05.307000,2024-10-19 18:03:51.406,1,20.0,False,,False,False,2024,United States Grand Prix
262,0 days 00:49:37.086000,ALB,23,0 days 00:01:41.402000,2,1.0,NaT,NaT,0 days 00:00:27.661000,0 days 00:00:39.894000,0 days 00:00:33.847000,0 days 00:48:23.355000,0 days 00:49:03.249000,0 days 00:49:37.096000,203.0,175.0,207.0,328.0,True,MEDIUM,2.0,True,Williams,0 days 00:47:55.684000,2024-10-19 18:05:41.783,1,20.0,False,,False,True,2024,United States Grand Prix
263,0 days 00:51:18.396000,ALB,23,0 days 00:01:41.310000,3,1.0,NaT,NaT,0 days 00:00:27.433000,0 days 00:00:40.128000,0 days 00:00:33.749000,0 days 00:50:04.529000,0 days 00:50:44.657000,0 days 00:51:18.406000,205.0,179.0,207.0,314.0,True,MEDIUM,3.0,True,Williams,0 days 00:49:37.086000,2024-10-19 18:07:23.185,1,20.0,False,,False,True,2024,United States Grand Prix
264,0 days 00:52:58.983000,ALB,23,0 days 00:01:40.587000,4,1.0,NaT,NaT,0 days 00:00:27.048000,0 days 00:00:39.945000,0 days 00:00:33.594000,0 days 00:51:45.454000,0 days 00:52:25.399000,0 days 00:52:58.993000,209.0,174.0,204.0,328.0,True,MEDIUM,4.0,True,Williams,0 days 00:51:18.396000,2024-10-19 18:09:04.495,1,20.0,False,,False,True,2024,United States Grand Prix
265,0 days 00:54:39.328000,ALB,23,0 days 00:01:40.345000,5,1.0,NaT,NaT,0 days 00:00:27.049000,0 days 00:00:39.982000,0 days 00:00:33.314000,0 days 00:53:26.042000,0 days 00:54:06.024000,0 days 00:54:39.338000,205.0,175.0,208.0,313.0,True,MEDIUM,5.0,True,Williams,0 days 00:52:58.983000,2024-10-19 18:10:45.082,1,20.0,False,,False,True,2024,United States Grand Prix


In [5]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [6]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [7]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [8]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
349,ALB,1,0.0,40.701,34.074,0,0.0,MEDIUM,2024,United States Grand Prix,0.0,0.0,11.0,17.0
350,ALO,1,0.0,41.927,33.899,0,0.0,MEDIUM,2024,United States Grand Prix,0.0,0.0,18.0,15.0
352,BOT,1,0.0,41.968,34.194,0,0.0,MEDIUM,2024,United States Grand Prix,0.0,0.0,20.0,19.0
344,COL,1,0.0,40.740,33.693,0,0.0,MEDIUM,2024,United States Grand Prix,0.0,0.0,12.0,10.0
346,GAS,1,0.0,41.024,34.037,0,0.0,MEDIUM,2024,United States Grand Prix,0.0,0.0,14.0,16.0


In [9]:
#2024
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="COL":
        return '#64C4FF'    
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    elif driver =="LAW":
        return '#6692FF'    
    else:
        return '#3860C6'

In [10]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,United States Grand Prix,2024,2,00:01:38.452000,98.452,RUS,MEDIUM,26.586,RUS,MEDIUM,38.586,SAI,MEDIUM,32.485,VER,MEDIUM
1,United States Grand Prix,2024,3,00:01:37.906000,97.906,VER,MEDIUM,26.414,NOR,MEDIUM,38.793,LEC,MEDIUM,32.308,VER,MEDIUM
2,United States Grand Prix,2024,4,00:01:38.150000,98.150,VER,MEDIUM,26.296,RUS,MEDIUM,38.886,NOR,MEDIUM,32.383,LEC,MEDIUM
3,United States Grand Prix,2024,5,00:01:38.329000,98.329,SAI,MEDIUM,26.352,LEC,MEDIUM,38.996,SAI,MEDIUM,32.638,NOR,MEDIUM
4,United States Grand Prix,2024,6,00:01:38.216000,98.216,LEC,MEDIUM,26.349,RUS,MEDIUM,38.954,RUS,MEDIUM,32.463,VER,MEDIUM


In [11]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,United States Grand Prix,2024,2,00:01:38.452000,98.452,RUS,MEDIUM,26.586,RUS,MEDIUM,38.586,SAI,MEDIUM,32.485,VER,MEDIUM,#27F4D2
1,United States Grand Prix,2024,3,00:01:37.906000,97.906,VER,MEDIUM,26.414,NOR,MEDIUM,38.793,LEC,MEDIUM,32.308,VER,MEDIUM,#3860C6
2,United States Grand Prix,2024,4,00:01:38.150000,98.150,VER,MEDIUM,26.296,RUS,MEDIUM,38.886,NOR,MEDIUM,32.383,LEC,MEDIUM,#3860C6
3,United States Grand Prix,2024,5,00:01:38.329000,98.329,SAI,MEDIUM,26.352,LEC,MEDIUM,38.996,SAI,MEDIUM,32.638,NOR,MEDIUM,#E80020
4,United States Grand Prix,2024,6,00:01:38.216000,98.216,LEC,MEDIUM,26.349,RUS,MEDIUM,38.954,RUS,MEDIUM,32.463,VER,MEDIUM,#E80020


In [12]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
17,United States Grand Prix,2024,19,00:01:37.463000,97.463,VER,MEDIUM,25.964,SAI,MEDIUM,38.978,PIA,MEDIUM,32.318,PIA,MEDIUM,#3860C6
16,United States Grand Prix,2024,18,00:01:37.613000,97.613,PIA,MEDIUM,26.027,LEC,MEDIUM,38.917,PIA,MEDIUM,32.388,PIA,MEDIUM,#FF8000
9,United States Grand Prix,2024,11,00:01:37.748000,97.748,LEC,MEDIUM,26.218,NOR,MEDIUM,39.098,LEC,MEDIUM,32.338,VER,MEDIUM,#E80020
15,United States Grand Prix,2024,17,00:01:37.752000,97.752,LEC,MEDIUM,26.177,HAM,MEDIUM,38.824,LEC,MEDIUM,32.352,PIA,MEDIUM,#E80020
10,United States Grand Prix,2024,12,00:01:37.835000,97.835,LEC,MEDIUM,26.082,LEC,MEDIUM,39.176,LEC,MEDIUM,32.537,VER,MEDIUM,#E80020


In [13]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=11)
    ))
 )

# fig2.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": LapTime_ordered.T.values
#                                 if c == "All"
#                                 else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )

fig2.update_layout(
    title="Sprint Race Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="Black"
    ),
    height=780,
    width=1200,
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)


fig2.show()

In [14]:
fig2.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Fastest Lap and Sectors/{}/{} {} Sprint Race Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year, year,gp),full_html=False, include_plotlyjs='cdn')

In [15]:
#Multi GP Plot

In [11]:
#2024
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="COL":
        return '#64C4FF'    
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [12]:
#2024
race_multi=['Austrian Grand Prix','Miami Grand Prix','Chinese Grand Prix']
#2023
# race_multi=['Austrian Grand Prix','Azerbaijan Grand Prix','Belgian Grand Prix','Qatar Grand Prix','São Paulo Grand Prix','United States Grand Prix']
#2022
# race_multi=['Austrian Grand Prix','Emilia Romagna Grand Prix','São Paulo Grand Prix']
#2021
# race_multi=['British Grand Prix','Italian Grand Prix','São Paulo Grand Prix']

year_multi=2024
for i in race_multi:

    df_bq=df_bq2[(df_bq2["GP"]==i)&(df_bq2["Year"]==year_multi)]
    GP=df_bq
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
    GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
    GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
    GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
    GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
    GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

    GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
    GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
    GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()
    GP_Rk=GP[GP["LapTime"]!="NaT"]
    GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
    GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
    GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)

    GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
    GP_Sectors=GP_Sectors.fillna(0)
    GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()
    Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
    Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
    Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
    Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
    Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
    Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
    Fast_Laps=Fast_Laps.reset_index(drop=True)
    Fast_Laps.head()
    Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
    Fast_Laps.head()
    LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
    LapTime_ordered.head()
    import plotly.graph_objects as go

    fig2 = go.Figure(go.Table
    (header={"values": LapTime_ordered.columns},
    cells=dict(
        values=LapTime_ordered.T.values,
        fill_color=[LapTime_ordered.color],
        align='center', font=dict(color='white', size=11)
        ))
    )
    fig2.update_layout(
        title="Fastest Lap and Sector Times Ordered By Lap Times For The {} {}".format(year_multi, i),
            title_x=0.5,
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=13,
            color="Black"
        ),
        height=780,
        width=1200,
        title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5),
    )
    fig2.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Fastest Lap and Sectors/{}/{} {} Sprint Race Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_10433/3493564426.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_10433/3493564426.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_10433/3493564426.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [161]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps.columns},
#   cells=dict(
#     values=Fast_Laps.T.values,
#     fill_color=[Fast_Laps.color],
#     align='center', font=dict(color='white', size=11)
#     ))
#  )

# # fig.update_layout(
# #     updatemenus=[
# #         {
# #             "buttons": [
# #                 {
# #                     "label": c,
# #                     "method": "update",
# #                     "args": [
# #                         {
# #                             "cells": {
# #                                 "values": Fast_Laps.T.values
# #                                 if c == "All"
# #                                 else Fast_Laps.loc[Fast_Laps["LapNumber"].eq(c)].T.values
# #                             }
# #                         }
# #                     ],
# #                 }
# #                 for c in ["All"] + Fast_Laps["LapNumber"].unique().tolist()
# #             ]
# #         }
# #     ]
# # )
# fig.update_layout(
#     title="Fastest Lap and Sector Times For Ordered By Lap Number",
#         title_x=0.5,
#         # template="plotly_dark",

#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=13,
#         color="Black"
#     ),
#     height=780,
#     width=1200,
#     title_font_family="PT Sans Narrow",
#     margin=dict(l=5, r=5, t=35, b=5),
# )

# fig.show()

In [162]:
# fig.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Number.html".format(year,gp))

In [163]:
# #get a shortened view for fastest drivers in the entire lap, S1, S2, and S3

# Fast_Laps_short=Fast_Laps[["GP","Year","LapNumber","Fastest Lap","Fastest S1","Fastest S2","Fastest S3","color"]].sort_values(by=["LapNumber"])
# Fast_Laps_short=Fast_Laps_short.reset_index(drop=True)
# Fast_Laps_short.head()

In [164]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps_short.columns},
#   cells=dict(
#     values=Fast_Laps_short.T.values,
#     height=50,
#     fill_color=[Fast_Laps_short.color],
#     align='center', font=dict(color='white', size=20)
#     ))
#  )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": Fast_Laps_short.T.values
#                                 if c == "All"
#                                 else Fast_Laps_short.loc[Fast_Laps_short["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + Fast_Laps_short["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )
# fig.update_layout(
#     title="Fastest Lap and Sector Times",
#         title_x=0.5,
#     yaxis = dict(tickfont = dict(size=20)),
#     xaxis = dict(tickfont = dict(size=20)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=20,
#         color="Black"
#     ),
#     height=2500,
#     title_font_family="PT Sans Narrow"
# )

# fig.show()

In [107]:
# #get a view for fastest lap in the entire race
# Fastest_Lap_Race=GP_Rk[GP_Rk["Race_RK"]==1][["Year","GP","Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound"]].reset_index(drop=True)
# Fastest_Lap_Race

In [108]:
# #Multiple plotly filters code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "y": 1 - (i / 5),
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "restyle",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df[menu].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df[menu].unique().tolist()
#             ],
#         }
#         for i, menu in enumerate(["Channel_type", "Date"])
#     ]
# )

In [109]:
# #Single Plotly filter code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df["Channel_type"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df["Channel_type"].unique().tolist()
#             ]
#         }
#     ]
# )